In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
pd.set_option('display.max_columns', None)


from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [23]:
df = pd.read_parquet('../data/treatments_2017-2020_train.parquet')

In [24]:
df.head()

,AGE,GENDER,RACE,ETHNIC,MARSTAT,EDUC,EMPLOY,VET,LIVARAG,PRIMINC,ARRESTS,DIVISION,SERVICES,DAYWAIT,PSOURCE,SUB1,FREQ1,FRSTUSE1,PSYPROB,FREQ_ATND_SELF_HELP,NOPRIOR,NUM_SUBS
0,2,Female,White,Not Hispanic,Never married,2,Part-time,Not veteran,Independent living,None,0,Pacific,Ambulatory,0,Individual,Opiates or Methadone,Daily use,3,No,No,0,1
1,2,Female,White,Not Hispanic,Never married,2,Not in labor force,Not veteran,Independent living,None,0,Pacific,Ambulatory,0,Other Health Care provider,Heroin,Daily use,4,No,No,1,2
2,1,Female,White,Not Hispanic,Never married,-1,Unemployed,Not veteran,Independent living,Public assistance,0,Pacific,Ambulatory,0,Individual,Heroin,Daily use,2,Yes,No,1,2
3,2,Female,White,Not Hispanic,Never married,2,Not in labor force,Not veteran,Homeless,None,0,Pacific,Ambulatory,0,Individual,Heroin,Daily use,3,No,No,1,2
4,3,Female,Alaska Native,Not Hispanic,Divorced/widowed,2,Not in labor force,Not veteran,Homeless,Other,0,Middle Atlantic,Ambulatory,-1,Other Health Care provider,Heroin,Daily use,4,Yes,No,1,3


In [25]:
df = df.sample(1000).reset_index(drop=True)

In [28]:
df.head()

,AGE,GENDER,RACE,ETHNIC,MARSTAT,EDUC,EMPLOY,VET,LIVARAG,PRIMINC,ARRESTS,DIVISION,SERVICES,DAYWAIT,PSOURCE,SUB1,FREQ1,FRSTUSE1,PSYPROB,FREQ_ATND_SELF_HELP,NOPRIOR,NUM_SUBS
0,1,Male,White,Not Hispanic,Separated,2,Unemployed,Not veteran,Independent living,None,0,South Atlantic,Detox,-1,Court,Methamphetamine,Daily use,2,No,Yes,1,3
1,4,Female,White,Not Hispanic,Divorced/widowed,2,Not in labor force,Not veteran,Independent living,"Retirement/pension, disability",0,New England,Detox,0,Individual,Heroin,Daily use,3,No,No,1,1
2,3,Male,White,Hispanic,Never married,3,Unemployed,Not veteran,Independent living,None,0,Middle Atlantic,Ambulatory,0,Court,Heroin,No use in the past month,1,Yes,Yes,0,2
3,4,Male,White,Not Hispanic,Divorced/widowed,0,Not in labor force,Not veteran,Independent living,Other,0,Middle Atlantic,Rehab/residential,-1,Alcohol/drug provider,Alcohol,No use in the past month,1,Yes,No,1,3
4,3,Male,Black,Not Hispanic,Divorced/widowed,2,Not in labor force,Not veteran,Independent living,None,0,New England,Ambulatory,-1,Individual,Heroin,Daily use,2,No,No,1,1


In [29]:
X = df.drop("NOPRIOR", axis=1)
y = df["NOPRIOR"]

In [31]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   AGE                  1000 non-null   int64 
 1   GENDER               1000 non-null   object
 2   RACE                 1000 non-null   object
 3   ETHNIC               1000 non-null   object
 4   MARSTAT              1000 non-null   object
 5   EDUC                 1000 non-null   int64 
 6   EMPLOY               1000 non-null   object
 7   VET                  1000 non-null   object
 8   LIVARAG              1000 non-null   object
 9   PRIMINC              1000 non-null   object
 10  ARRESTS              1000 non-null   int64 
 11  DIVISION             1000 non-null   object
 12  SERVICES             1000 non-null   object
 13  DAYWAIT              1000 non-null   int64 
 14  PSOURCE              1000 non-null   object
 15  SUB1                 1000 non-null   object
 16  FREQ1  

In [33]:
columns_one_hot = [
    "GENDER",
    "RACE",
    "ETHNIC",
    "MARSTAT",
    "EMPLOY",
    "VET",
    "LIVARAG",
    "PRIMINC",
    "DIVISION",
    "SERVICES",
    "PSOURCE",
    "SUB1",
    "FREQ1",
    "PSYPROB",
    "FREQ_ATND_SELF_HELP",
]

In [34]:
X = pd.get_dummies(X, columns=columns_one_hot, drop_first=True)

In [35]:
X

,AGE,EDUC,ARRESTS,DAYWAIT,FRSTUSE1,NUM_SUBS,GENDER_Male,RACE_American Indian,RACE_Asian,RACE_Black,"RACE_Hawaii, Pacific Islander",RACE_Other race,RACE_Two or more races,RACE_White,ETHNIC_Not Hispanic,MARSTAT_Never married,MARSTAT_Now married,MARSTAT_Separated,EMPLOY_Not in labor force,EMPLOY_Part-time,EMPLOY_Unemployed,VET_Veteran,LIVARAG_Homeless,LIVARAG_Independent living,PRIMINC_Other,PRIMINC_Public assistance,"PRIMINC_Retirement/pension, disability",PRIMINC_Wages/salary,DIVISION_East South Central,DIVISION_Middle Atlantic,DIVISION_Mountain,DIVISION_New England,DIVISION_Pacific,DIVISION_South Atlantic,DIVISION_U.S. territories,DIVISION_West North Central,DIVISION_West South Central,SERVICES_Detox,SERVICES_Rehab/residential,PSOURCE_Community,PSOURCE_Court,PSOURCE_Employer,PSOURCE_Individual,PSOURCE_Other Health Care provider,PSOURCE_School,SUB1_Benzodiazepines or Tranquilizers,SUB1_Cocaine/Crack,SUB1_Heroin,SUB1_Marijuana/Hashish,SUB1_Methamphetamine,SUB1_Opiates or Methadone,SUB1_Other,SUB1_Other Amphetamines or Stimulants,SUB1_PCP or Other Hallucinogens,FREQ1_No use in the past month,FREQ1_Some use,PSYPROB_Yes,FREQ_ATND_SELF_HELP_Yes
0,1,2,0,-1,2,3,True,False,False,False,False,False,False,True,True,False,False,True,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True
1,4,2,0,0,3,1,False,False,False,False,False,False,False,True,True,False,False,False,True,False,False,False,False,True,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
2,3,3,0,0,1,2,True,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,True,True
3,4,0,0,-1,1,3,True,False,False,False,False,False,False,True,True,False,False,False,True,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False
4,3,2,0,-1,2,1,True,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3,2,0,0,4,1,True,False,False,False,False,False,False,True,True,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,True,True
996,3,2,0,0,2,3,True,False,False,False,False,False,False,True,True,True,False,False,False,False,True,False,False,True,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
997,4,2,0,0,1,2,False,False,False,True,False,False,False,False,True,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
998,2,1,0,0,1,3,True,False,False,False,False,False,False,True,True,T

In [36]:
y

0      1
1      1
2      0
3      1
4      1
      ..
995    1
996    1
997    1
998    1
999    0
Name: NOPRIOR, Length: 1000, dtype: int64

In [37]:


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [38]:
X_train.shape

(700, 58)

In [39]:



rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

RandomForestClassifier()

In [40]:
rfc_predict = rfc.predict(X_test)

In [43]:
rfc_cv_score = cross_val_score(rfc, X, y, cv=10, scoring='roc_auc')

In [44]:
rfc_cv_score

array([0.67267267, 0.74946375, 0.65529816, 0.74860575, 0.75375375,
       0.6996997 , 0.63709677, 0.65747029, 0.74002547, 0.69779287])

In [45]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[ 50  50]
 [ 42 158]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.54      0.50      0.52       100
           1       0.76      0.79      0.77       200

    accuracy                           0.69       300
   macro avg       0.65      0.65      0.65       300
weighted avg       0.69      0.69      0.69       300



=== All AUC Scores ===
[0.67267267 0.74946375 0.65529816 0.74860575 0.75375375 0.6996997
 0.63709677 0.65747029 0.74002547 0.69779287]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.7011879178475104


In [51]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np   
import pandas as pd  

import matplotlib.pyplot as plt

import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


In [54]:
from sklearn.model_selection import KFold
pipelines = []
pipelines.append(('LogisticRegression' , (Pipeline([('LR' ,LogisticRegression())]))))
pipelines.append(('SVC' , (Pipeline([('SVC' ,LinearSVC())]))))
#pipelines.append(('MultinomailNB' , (Pipeline([('MNB' ,MultinomialNB())]))))
pipelines.append(('RandomForest' , (Pipeline([('RF' ,RandomForestClassifier())]))))
pipelines.append(('XGBoost' , (Pipeline([('XGB' ,XGBClassifier())]))))

In [55]:
model_name = []
results = []
for pipe ,model in pipelines:
    kfold = KFold(n_splits=10)
    cross_validation_results = cross_val_score(model , X ,y ,cv =kfold , scoring='accuracy')
    results.append(cross_validation_results)
    model_name.append(pipe)
    msg = "%s: 'mean accuracy :' %f  'std deviation :' (%f)"  \
          % (model[0],  cross_validation_results.mean(),  cross_validation_results.std())

    print(msg)

LogisticRegression(): 'mean accuracy :' 0.687000  'std deviation :' (0.037430)
LinearSVC(): 'mean accuracy :' 0.686000  'std deviation :' (0.032924)
RandomForestClassifier(): 'mean accuracy :' 0.687000  'std deviation :' (0.056577)


In [ ]:
results_mean = np.mean(np.array(results),axis=1)


In [ ]:
baseline_models = pd.DataFrame(list(zip(model_name,results_mean)), columns=['Model','Accuracy'])


In [ ]:
baseline_models
